In [ ]:
import sys
import os
import datetime
import time
import math

import numpy as np
import skimage as sk
from skimage import io
import skimage.measure
import skimage.morphology
import sklearn.metrics
import pandas as pd
import h5py

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
results_df = pd.read_csv("results_G1/exp.csv", sep=",")
# results_df = pd.read_csv("results_G4/exp.csv", sep=",")
# results_df = pd.read_csv("results_G1_full_predict/exp.csv", sep=",")
# results_df = pd.read_csv("results_G1_input_conv_full_predict/exp.csv", sep=",")
# results_df = pd.read_csv("results_G1_HA1_model/exp.csv", sep=",")

In [ ]:
results_df['modelname'] = results_df['modelname'].map(lambda x: str(x)[:-11])

In [ ]:
results_df = results_df.replace("2Dvs25D_256-1-1-1", "I3_1SL   ")
results_df = results_df.replace("2Dvs25D_256-1-3-1", "I3_3SL   ")
results_df = results_df.replace("2Dvs25D_256-1-5-1", "I3_5SL   ")
results_df = results_df.replace("2Dvs25D_256-1-3-3", "I3_3SL_G ")
results_df = results_df.replace("2Dvs25D_256-1-5-5", "I3_5SL_G ")

results_df = results_df.replace("2Dvs25D_256-2-1-1", "LW4_1SL  ")
results_df = results_df.replace("2Dvs25D_256-2-3-1", "LW4_3SL  ")
results_df = results_df.replace("2Dvs25D_256-2-5-1", "LW4_5SL  ")
results_df = results_df.replace("2Dvs25D_256-2-3-3", "LW4_3SL_G")
results_df = results_df.replace("2Dvs25D_256-2-5-5", "LW4_5SL_G")

In [ ]:
results_df.groupby(["modelname"]).count()

In [ ]:
results_np = results_df.to_numpy()

f1c1_valid_mean = []
f1c1_valid_std = []
f1c2_valid_mean = []
f1c2_valid_std = []
f1c3_valid_mean = []
f1c3_valid_std = []
f1c1_test_mean = []
f1c1_test_std = []
f1c2_test_mean = []
f1c2_test_std = []
f1c3_test_mean = []
f1c3_test_std = []
f1_mean_sum_valid = []
f1_mean_sum_test = []
    
for model in range(results_np.shape[0]):
    # F1 Valid mean/std
    f1c1 = []
    f1c2 = []
    f1c3 = []
    
    for i in range(0,10):
        if results_np[model,3*i+1] + results_np[model,3*i+2] + results_np[model,3*i+3] > 0:
            f1c1.append(results_np[model,3*i+1]) 
            f1c2.append(results_np[model,3*i+2])
            f1c3.append(results_np[model,3*i+3])
            
    f1c1 = np.array(f1c1)
    f1c2 = np.array(f1c2)
    f1c3 = np.array(f1c3)
    
    f1c1_valid_mean.append(f1c1.mean())
    f1c1_valid_std.append(f1c1.std())
    f1c2_valid_mean.append(f1c2.mean())
    f1c2_valid_std.append(f1c2.std())
    f1c3_valid_mean.append(f1c3.mean())
    f1c3_valid_std.append(f1c3.std())
    f1_mean_sum_valid.append(f1c1.mean()+f1c2.mean()+f1c3.mean())
    
    # F1 Test mean/std
    f1c1 = []
    f1c2 = []
    f1c3 = []
    
    for i in range(0,10):
        if results_np[model,30+3*i+1] + results_np[model,30+3*i+2] + results_np[model,30+3*i+3] > 0:
            f1c1.append(results_np[model,30+3*i+1]) 
            f1c2.append(results_np[model,30+3*i+2])
            f1c3.append(results_np[model,30+3*i+3])
        
    f1c1 = np.array(f1c1)
    f1c2 = np.array(f1c2)
    f1c3 = np.array(f1c3)
    
    f1c1_test_mean.append(f1c1.mean())
    f1c1_test_std.append(f1c1.std())
    f1c2_test_mean.append(f1c2.mean())
    f1c2_test_std.append(f1c2.std())
    f1c3_test_mean.append(f1c3.mean())
    f1c3_test_std.append(f1c3.std())
    f1_mean_sum_test.append(f1c1.mean()+f1c2.mean()+f1c3.mean())
    
results_df['F1_C1_V_MEAN'] = f1c1_valid_mean
results_df['F1_C1_V_STD'] = f1c1_valid_std
results_df['F1_C2_V_MEAN'] = f1c2_valid_mean
results_df['F1_C2_V_STD'] = f1c2_valid_std
results_df['F1_C3_V_MEAN'] = f1c3_valid_mean
results_df['F1_C3_V_STD'] = f1c3_valid_std
results_df['F1_C1_T_MEAN'] = f1c1_test_mean
results_df['F1_C1_T_STD'] = f1c1_test_std
results_df['F1_C2_T_MEAN'] = f1c2_test_mean
results_df['F1_C2_T_STD'] = f1c2_test_std
results_df['F1_C3_T_MEAN'] = f1c3_test_mean
results_df['F1_C3_T_STD'] = f1c3_test_std

results_df['F1_MEAN_SUM_V'] = f1_mean_sum_valid
results_df['F1_MEAN_SUM_T'] = f1_mean_sum_test

In [ ]:
results_df.nlargest(2,'F1_MEAN_SUM_T')

In [ ]:
for cl in ['F1_C1_T_MEAN', 'F1_C2_T_MEAN', 'F1_C3_T_MEAN']:
    for modelname in sorted(results_df.modelname.unique()):
        print(
            modelname, cl, ' \t',
            format(results_df.loc[results_df['modelname'] == modelname].nlargest(8,'F1_MEAN_SUM_V')[[cl]].to_numpy().mean(), ".3f"),
            format(results_df.loc[results_df['modelname'] == modelname].nlargest(8,'F1_MEAN_SUM_V')[[cl]].to_numpy().std(), ".3f")
            # format(results_df.loc[results_df['modelname'] == modelname].nlargest(8,'F1_MEAN_SUM_V')[[cl]].to_numpy().max(), ".3f")
        )

|("results_G1/exp.csv", sep=",")| I3 C1 | I3 C2 | I3 C3 | LW4 C1 | LW4 C2 | LW4 C3 |
| - | - | - | - | - | - | - |
| 1SL    | **0.940** 0.012 | **0.791** 0.022 | **0.742** 0.009 | **0.954** 0.003 | **0.752** 0.007 | **0.734** 0.011 |       
| 3SL    | *0.782* 0.100 | *0.646* 0.055 | 0.536 0.046 | **0.953** 0.002 | 0.686 0.013 | 0.700 0.018 |     
| 3SL_G  | 0.864 0.046 | 0.708 0.043 | 0.510 0.068 | **0.953** 0.003 | 0.683 0.015 | 0.701 0.020 |      
| 5SL    | *0.783* 0.071 | *0.619* 0.053 | 0.500 0.074 | **0.949** 0.003 | 0.694 0.008 | 0.700 0.016 |     
| 5SL_G  | 0.860 0.061 | 0.746 0.037 | 0.558 0.050 | **0.951** 0.004 | 0.686 0.012 | 0.706 0.010 |      

|("results_G4/exp.csv", sep=",")| I3 C1 | I3 C2 | I3 C3 | LW4 C1 | LW4 C2 | LW4 C3 |
| - | - | - | - | - | - | - |
| 1SL    | **0.945** 0.010 | **0.798** 0.020 | **0.744** 0.007 | **0.954** 0.003 | **0.755** 0.005 | **0.741** 0.005 |       
| 3SL    | 0.808 0.049 | 0.708 0.053 | 0.532 0.069 | **0.954** 0.003 | 0.676 0.014 | 0.694 0.022 |     
| 3SL_G  | 0.828 0.053 | 0.708 0.046 | 0.496 0.056 | **0.953** 0.003 | 0.691 0.010 | 0.697 0.016 |      
| 5SL    | 0.790 0.075 | 0.590 0.048 | 0.493 0.077 | **0.951** 0.003 | 0.693 0.006 | 0.709 0.008 |     
| 5SL_G  | 0.840 0.039 | 0.687 0.040 | 0.550 0.016 | **0.952** 0.003 | 0.681 0.014 | 0.698 0.014 |     

|("results_G1_full_predict/exp.csv", sep=",")| I3 C1 | I3 C2 | I3 C3 | LW4 C1 | LW4 C2 | LW4 C3 |
| - | - | - | - | - | - | - |
| 1SL    | **0.948** 0.008 | **0.785** 0.019 | **0.739** 0.009 | **0.949** 0.009 | **0.748** 0.006 | **0.737** 0.005 |       
| 3SL    | 0.821 0.050 | 0.676 0.042 | 0.525 0.069 | **0.950** 0.006 | 0.682 0.013 | 0.707 0.016 |     
| 3SL_G  | 0.865 0.041 | 0.712 0.062 | 0.526 0.060 | **0.951** 0.004 | 0.690 0.013 | 0.696 0.020 |      
| 5SL    | 0.818 0.068 | 0.677 0.048 | 0.539 0.035 | **0.949** 0.006 | 0.689 0.010 | 0.703 0.015 |     
| 5SL_G  | 0.804 0.056 | 0.691 0.041 | 0.519 0.051 | **0.950** 0.005 | 0.681 0.009 | 0.698 0.021 |       

|("results_G1_input_conv_full_predict/exp.csv", sep=",")| I3 C1 | I3 C2 | I3 C3 | LW4 C1 | LW4 C2 | LW4 C3 |
| - | - | - | - | - | - | - |
| 1SL    | **0.941 **0.011 | **0.808** 0.010 | **0.744** 0.008 | **0.954** 0.002 | **0.749** 0.006 | **0.742** 0.004 |       
| 3SL    | 0.877 0.015 | 0.722 0.049 | 0.572 0.039 | **0.947** 0.007 | 0.679 0.014 | 0.706 0.010 |     
| 3SL_G  | 0.827 0.027 | 0.680 0.055 | 0.535 0.037 | **0.953** 0.003 | 0.687 0.013 | 0.713 0.008 |      
| 5SL    | *0.773* 0.059 | *0.628* 0.064 | 0.541 0.045 | **0.946** 0.008 | 0.682 0.016 | 0.694 0.018 |     
| 5SL_G  | 0.822 0.054 | 0.678 0.053 | *0.487* 0.070 | **0.949** 0.005 | 0.679 0.011 | 0.707 0.005 |     


|("results_G1_HA1_model/exp.csv", sep=",")| I3 C1 | I3 C2 | I3 C3 | LW4 C1 | LW4 C2 | LW4 C3 |
| - | - | - | - | - | - | - |
| 1SL    | **0.965** 0.003 | **0.825** 0.005 | **0.777** 0.014 | **0.960** 0.001 | **0.772** 0.006 | 0.749 0.008 |       
| 3SL    | **0.953** 0.009 | 0.791 0.018 | 0.634 0.048 | **0.957** 0.003 | 0.745 0.008 | **0.762** 0.006 |     
| 5SL    | 0.942 0.013 | 0.745 0.039 | 0.677 0.040 | **0.957** 0.002 | 0.743 0.007 | **0.768** 0.005 |     
